In [20]:

from pybliometrics.scopus import AbstractRetrieval

In [112]:
ab = AbstractRetrieval("10.1016/j.softx.2019.100263")
print(ab.title)
print(ab.authors)

pybliometrics: Scriptable bibliometrics using a Python interface to Scopus
[Author(auid=57209617104, indexed_name='Rose M.E.', surname='Rose', given_name='Michael E.', affiliation='60105007'), Author(auid=7004212771, indexed_name='Kitchin J.R.', surname='Kitchin', given_name='John R.', affiliation='60027950')]


In [ ]:
# "AuthorRetrieval" from pybliometrics package
author_id = 6603277503
au = AuthorRetrieval(author_id)
print(au)

In [229]:
au.affiliation_current

[Affiliation(id=112554118, parent=60023561, type='dept', relationship='author', afdispname=None, preferred_name='Department of Medicine', parent_preferred_name='Centre Hospitalier Universitaire Vaudois', country_code='che', country='Switzerland', address_part='Rue de Bugnon 21', city='Lausanne', state='VD', postal_code='1011', org_domain='lausanneuniversityhospital', org_URL='https://www.lausanneuniversityhospital.com/home')]

In [115]:
docs = pd.DataFrame(au.get_documents(refresh=10))
docs.columns

Index(['eid', 'doi', 'pii', 'pubmed_id', 'title', 'subtype',
       'subtypeDescription', 'creator', 'afid', 'affilname',
       'affiliation_city', 'affiliation_country', 'author_count',
       'author_names', 'author_ids', 'author_afids', 'coverDate',
       'coverDisplayDate', 'publicationName', 'issn', 'source_id', 'eIssn',
       'aggregationType', 'volume', 'issueIdentifier', 'article_number',
       'pageRange', 'description', 'authkeywords', 'citedby_count',
       'openaccess', 'freetoread', 'freetoreadLabel', 'fund_acr', 'fund_no',
       'fund_sponsor'],
      dtype='object')

In [6]:
docs.coverDate

0      2022-12-01
1      2022-12-01
2      2022-11-03
3      2022-10-01
4      2022-10-01
          ...    
167    1998-03-07
168    1997-01-01
169    1995-01-01
170    1994-01-01
171    1994-01-01
Name: coverDate, Length: 172, dtype: object

In [116]:
# Delete documents that are too old
new_docs = docs[(docs.coverDate > '2018')]

In [117]:
new_docs["author_count"]

0      97
1     100
2     100
3      93
4      94
     ... 
59     19
60     63
61     13
62     60
63     63
Name: author_count, Length: 64, dtype: object

In [118]:
new_docs["new_author_count"] = [int(i) for i in new_docs.author_count]
new_docs["new_author_count"][0]

/tmp/ipykernel_20986/2511647989.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_docs["new_author_count"] = [int(i) for i in new_docs.author_count]


97

In [119]:
my_docs = new_docs

In [94]:
# Delete documents with too many co-authorship relationships
my_docs = new_docs[(new_docs.new_author_count < 30)]

In [95]:
my_docs.reset_index(drop=True, inplace=True)

In [120]:
len(my_docs)

64

In [121]:
# Get all the author info in the docs of one researcher
au_id = my_docs.author_ids
au_id

0     57205100307;35069025400;9237623500;56146581800...
1     56653485300;55636643100;57226842887;5783073580...
2     56340274300;57193500089;6507087260;15753450100...
3     56735961400;57193523588;6506304249;8718959100;...
4     57200984619;56469814200;57358926700;8673143300...
                            ...                        
59    57201479444;25651390500;57208011398;3611405450...
60    57189377649;6506014076;7004596986;7004121246;6...
61    7801624038;57208011398;57197467935;26427291000...
62    7006795092;7006085120;7103380067;19835874200;3...
63    6701433181;6602489733;35742245500;35372664600;...
Name: author_ids, Length: 64, dtype: object

In [122]:
def find_coauthors(au_id):
    au = AuthorRetrieval(au_id)
    docs = pd.DataFrame(au.get_documents(refresh=10))
    # Delete documents that are too old
    new_docs = docs[(docs.coverDate > '2018')]
    # Delete documents with too many co-authorship relationships
    new_docs["new_author_count"] = [int(i) for i in new_docs.author_count]
    my_docs = new_docs[(new_docs.new_author_count < 10)]
    
    my_docs.reset_index(drop=True, inplace=True)

def get_hindex(au_id):
    au = AuthorRetrieval(au_id)
    return au.h_index
def get_auname(au_id):
    au = AuthorRetrieval(au_id)
    return au.given_name+' '+au.surname

In [123]:
# Obtain NL co-authors
coau_id2 = []
link2 = []
all_node2 = []
# group2 = {}
# team2 = 0
for i in au_id:
    coau_id2 = i.split(";")
    new_coau_id2 = []
    for j in coau_id2:
        aff = AuthorRetrieval(j).affiliation_current
        if AuthorRetrieval(j).affiliation_current:
            # Geo-filtering 
            if AuthorRetrieval(j).affiliation_current[0][7] == 'nld':
                all_node2.append(j)
                new_coau_id2.append(j)
     #   group[j] = team
    link2 = link2+list(itertools.combinations(new_coau_id2, 2))

In [232]:
coau_id2

['57205100307',
 '35069025400',
 '9237623500',
 '56146581800',
 '24172784400',
 '57905194900',
 '57904184400',
 '55324449300',
 '56735961400',
 '37030466200',
 '22957488700',
 '7102045384',
 '24174793000',
 '7102575527',
 '6603304614',
 '7003684255',
 '55880489000',
 '8718959100',
 '18533651800',
 '57220123078',
 '7201383500',
 '22933317900',
 '35745452800',
 '8534725900',
 '7402340201',
 '6507264676',
 '57197725367',
 '57195968514',
 '6603277503',
 '57211478548',
 '35309243900',
 '16142826900',
 '6504624988',
 '55978666700',
 '7004627344',
 '26430969300',
 '12776997200',
 '6603883785',
 '57206774445',
 '7003525887',
 '7005051628',
 '6506575476',
 '6506296272',
 '57831865800',
 '22984831600',
 '7006620847',
 '7202350941',
 '57521975900',
 '7202406848',
 '57220123784',
 '6603816559',
 '6602186102',
 '6701349829',
 '57193216712',
 '57193523588',
 '56764357800',
 '16033216900',
 '14022858600',
 '6506992865',
 '35964694300',
 '36770934000',
 '10139630600',
 '56690563900',
 '57220118224',
 

In [132]:
def getCombinations(seq):
    combinations = list()
    seq = seq.sort()
    for i in range(0,len(seq)):
        for j in range(i+1,len(seq)):
            combinations.append([seq[i],seq[j]])
    return combinations

In [163]:
AuthorRetrieval(author_id).affiliation_current

[Affiliation(id=112554118, parent=60023561, type='dept', relationship='author', afdispname=None, preferred_name='Department of Medicine', parent_preferred_name='Centre Hospitalier Universitaire Vaudois', country_code='che', country='Switzerland', address_part='Rue de Bugnon 21', city='Lausanne', state='VD', postal_code='1011', org_domain='lausanneuniversityhospital', org_URL='https://www.lausanneuniversityhospital.com/home')]

In [213]:
author_id = 6603277503
node_retrieved = []

def nld_coauthor(author_id, depth, node_retrieved):
    au = AuthorRetrieval(author_id)
    docs = pd.DataFrame(au.get_documents())
    # Access to documents for the last five years
    new_docs = docs[(docs.coverDate > '2018')]
    au_id = new_docs.author_ids
    
    link = []
    all_node = []

    node_retrieved.append(author_id)
    
    for i in au_id:
        coau_id = i.split(";")
        coau_id = list(map(int, coau_id))
        new_coau_id = []
        for j in coau_id:
            aff = AuthorRetrieval(j).affiliation_current
            if AuthorRetrieval(j).affiliation_current:
                # Geo-filtering 
                if AuthorRetrieval(j).affiliation_current[0][7] == 'nld':
                    if j not in all_node:
                        all_node.append(j)
                    new_coau_id.append(j)
                    
        sorted_new_coauid = list(map(int, new_coau_id))
        sorted_new_coauid.sort()
        link = link+list(itertools.combinations(sorted_new_coauid, 2))
        # Do recursion (increase depth of the network)
        if depth > 0:
            for j in sorted_new_coauid:
                if j not in node_retrieved:
                    nld_coauthor(j, depth-1)
    return all_node, link
            
        

In [228]:
# slow
x,y = nld_coauthor(author_id, 1, node_retrieved)
print(x)

TypeError: nld_coauthor() missing 1 required positional argument: 'node_retrieved'

In [224]:
x2,y2 = nld_coauthor(57203646157, 0, node_retrieved)


In [227]:
print(x2)

[57203646157, 57221791778, 7005930079, 7006279060, 57537265700, 57218949992, 35378641700, 6701804604, 24075711800, 8050973500, 7005763178, 24528146600, 55840875500, 57003590900, 14219583800, 57214743523, 57202056973, 6602187919]


In [ ]:
# Get all the subject areas of every node
counter_node2 = Counter(all_node2)
areas2 = []
for i in counter_node2:
    au_info2 = AuthorRetrieval(i)
    subjects2 = au_info2.subject_areas
    areas2.append(subjects2)
areas2

In [197]:
docs.coverDate

0      2022-12-01
1      2022-12-01
2      2022-11-03
3      2022-10-01
4      2022-10-01
          ...    
167    1998-03-07
168    1997-01-01
169    1995-01-01
170    1994-01-01
171    1994-01-01
Name: coverDate, Length: 172, dtype: object

In [205]:
new_au = AuthorRetrieval(6603277503)
new_docs =pd.DataFrame(new_au.get_documents())
#new_docs = pd.DataFrame(new_au.get_documents(refresh=10))
#new_coauthors = pd.DataFrame(new_au.get_coauthors())
#print(new_au.given_name+' '+new_au.surname)

In [127]:
counter_node2 = Counter(all_node2)
areas2 = []
for i in counter_node2:
    au_info2 = AuthorRetrieval(i)
    subjects2 = au_info2.subject_areas
    areas2.append(subjects2)
areas2

[[Subjectarea(area='Sociology and Political Science', abbreviation='SOCI', code=3312),
  Subjectarea(area='Neurology (clinical)', abbreviation='MEDI', code=2728),
  Subjectarea(area='Life-span and Life-course Studies', abbreviation='SOCI', code=3319),
  Subjectarea(area='Multidisciplinary', abbreviation='MULT', code=1000),
  Subjectarea(area='Neuropsychology and Physiological Psychology', abbreviation='PSYC', code=3206),
  Subjectarea(area='Medicine (miscellaneous)', abbreviation='MEDI', code=2701),
  Subjectarea(area='Behavioral Neuroscience', abbreviation='NEUR', code=2802),
  Subjectarea(area='Pediatrics, Perinatology and Child Health', abbreviation='MEDI', code=2735),
  Subjectarea(area='Psychiatry and Mental Health', abbreviation='MEDI', code=2738),
  Subjectarea(area='Molecular Biology', abbreviation='BIOC', code=1312),
  Subjectarea(area='Developmental and Educational Psychology', abbreviation='PSYC', code=3204),
  Subjectarea(area='Physiology', abbreviation='BIOC', code=1314),


In [128]:
all_authors = list(set(all_node2))
len(all_authors)

6

In [129]:
au_subject2 = []
for i in range(len(areas2)):
    subject_list2 = []
    for j in areas2[i]:
        subject_list2.append(j.abbreviation)
    au_subject2.append(Counter(subject_list2).most_common(1)[0][0])
    
list(set(au_subject2))

['MEDI', 'MULT']

In [98]:
# Obtain co-authors
coau_id = []
link = []
all_node = []
group = {}
team = 0
for i in au_id:
    coau_id = i.split(";")
    new_coau_id = []
    for j in coau_id:
        aff = AuthorRetrieval(j).affiliation_current
        if AuthorRetrieval(j).affiliation_current:
            #if AuthorRetrieval(j).affiliation_current[0][7] == 'nld':
                all_node.append(j)
                new_coau_id.append(j)
     #   group[j] = team
    link = link+list(itertools.combinations(new_coau_id, 2))
    team = team+1

In [99]:
len(all_node)

363

In [100]:
counter_node = Counter(all_node)
areas = []
for i in counter_node:
    au_info = AuthorRetrieval(i)
    subjects = au_info.subject_areas
    areas.append(subjects)
areas

[[Subjectarea(area='Immunology and Microbiology (all)', abbreviation='IMMU', code=2400),
  Subjectarea(area='Immunology and Allergy', abbreviation='MEDI', code=2723),
  Subjectarea(area='Pediatrics, Perinatology and Child Health', abbreviation='MEDI', code=2735),
  Subjectarea(area='Multidisciplinary', abbreviation='MULT', code=1000),
  Subjectarea(area='Internal Medicine', abbreviation='MEDI', code=2724),
  Subjectarea(area='Infectious Diseases', abbreviation='MEDI', code=2725),
  Subjectarea(area='Radiology, Nuclear Medicine and Imaging', abbreviation='MEDI', code=2741),
  Subjectarea(area='Pharmacology (medical)', abbreviation='MEDI', code=2736),
  Subjectarea(area='Biochemistry, Genetics and Molecular Biology (all)', abbreviation='BIOC', code=1300),
  Subjectarea(area='Pharmacology', abbreviation='PHAR', code=3004),
  Subjectarea(area='Applied Microbiology and Biotechnology', abbreviation='IMMU', code=2402),
  Subjectarea(area='Obstetrics and Gynecology', abbreviation='MEDI', code=

In [105]:
au_subject = []
for i in range(len(areas)):
    subject_list = []
    for j in areas[i]:
        subject_list.append(j.abbreviation)
    au_subject.append(Counter(subject_list).most_common(1)[0][0])

In [106]:
au_areas = {}
for n in range(len(all_authors)):
    au_areas[all_authors[n]] = au_subject[n]

In [107]:
# Assign groups
exist_area = {}
au_group = {}
group_num = 0
for i in au_areas:
    if au_areas[i] not in exist_area:
        exist_area[au_areas[i]] = group_num
        au_group[i] = group_num
        group_num += 1
    else:
        au_group[i] = exist_area[au_areas[i]]
        

In [108]:
# Initialize nextworkx
GG = nx.Graph()
GG.add_edges_from(link)

In [109]:
sources = []
targets = []
weights = []

for i in link:
    sources.append(i[0])
    targets.append(i[1])
    weights.append(link.count(i))

In [238]:
# Pyvis network
N = Network(height=800, width="100%", bgcolor="#222222", font_color="white", select_menu=True, notebook = True)
N.toggle_hide_edges_on_drag(False)
N.barnes_hut()

edge_data = zip(sources, targets, weights)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    N.add_node(src, src, title=src, group = au_group[src])
    N.add_node(dst, dst, title=dst, group = au_group[dst])
    N.add_edge(src, dst, value=w)

neighbor_map = N.get_adj_list()

# add neighbor data to node hover data
for node in N.nodes:
    neighbors = []
    for neighbor_id in neighbor_map[node["id"]]:
        neigh = AuthorRetrieval(neighbor_id)
        neighbors.append(neigh.given_name+' '+neigh.surname)
    #node["title"] = " Neighbors: \n" + " \n".join(neighbors)
    node["title"] = "Link to the author’s API page:\n" + AuthorRetrieval(node["id"]).self_link
    node["value"] = get_hindex(node["id"])
    node["label"] = AuthorRetrieval(node["id"]).given_name+' '+AuthorRetrieval(node["id"]).surname
    # node["group"] = group[node["id"]]
    # node["font_size"] = get_hindex(node["id"])

N.show("scopus226.html")

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [ ]:
G = nx.DiGraph()

edge_data = zip(sources, targets, weights)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    G.add_node(src, src, title=src, group = au_group[src])
    G.add_node(dst, dst, title=dst, group = au_group[dst])
    G.add_edge(src, dst, value=w)

neighbor_map = G.get_adj_list()

# add neighbor data to node hover data
for node in G.nodes:
    neighbors = []
    for neighbor_id in neighbor_map[node["id"]]:
        neigh = AuthorRetrieval(neighbor_id)
        neighbors.append(neigh.given_name+' '+neigh.surname)
    #node["title"] = " Neighbors: \n" + " \n".join(neighbors)
    node["title"] = "Link to the author’s API page:\n" + AuthorRetrieval(node["id"]).self_link
    node["value"] = get_hindex(node["id"])
    node["label"] = AuthorRetrieval(node["id"]).given_name+' '+AuthorRetrieval(node["id"]).surname)

nx.write_gexf(G, 'scopus1078.gexf')

In [ ]:
def build_graph(G, au_id, depth, bredth, last_author=[]):
    author_search = scholarly.search_author_id(sch_id)
    author = scholarly.fill(author_search, sections=['coauthors','indices','counts'])
    author_name = author['name']
    author_hin = author['hindex']
    author_ncite = author['citedby']
    G.add_node(author_name, hindex=author_hin, name=author_name, ncite=author_ncite)
    print("now at %s, depth=%d"%(author_name, depth))
    if depth>0:
        for coauthor in author['coauthors'][:int(bredth)]:
            G.add_edge(author_name,coauthor['name'])
            if coauthor['name'] not in last_author:
                last_author.append(author_name)
                build_graph(G, coauthor['scholar_id'], depth-1, bredth, last_author)

In [ ]:
pos = nx.spring_layout(GG)

plt.figure(figsize=(100,80))

sizes = [ 20*GG.degree[node]+5 for node in GG.nodes]
dolphin_pr = nx.pagerank(GG)
colors = [ dolphin_pr[node] for node in GG.nodes]

pos = nx.spring_layout(GG)
s = nx.draw_networkx_nodes(GG, pos, node_size=sizes, node_color= colors,cmap='inferno')
cbar = plt.colorbar(s)
nx.draw_networkx_edges(GG, pos, edge_color='#4682B4')

#label top ten of centrality
centrality = dict(GG.degree())
degtop = sorted(centrality.keys(), key= lambda x:-centrality[x])[:50]
new_labels = {node:node for node in degtop}
nx.draw_networkx_labels(GG, pos, labels=new_labels, font_size=50)


plt.title('Mapping coauthorship', size = 50);
plt.axis('off');

In [ ]:
def build_graph(G, au_id, depth, bredth, last_author=[]):
    author_search = scholarly.search_author_id(sch_id)
    author = scholarly.fill(author_search, sections=['coauthors','indices','counts'])
    author_name = author['name']
    author_hin = author['hindex']
    author_ncite = author['citedby']
    G.add_node(author_name, hindex=author_hin, name=author_name, ncite=author_ncite)
    print("now at %s, depth=%d"%(author_name, depth))
    if depth>0:
        for coauthor in author['coauthors'][:int(bredth)]:
            G.add_edge(author_name,coauthor['name'])
            if coauthor['name'] not in last_author:
                last_author.append(author_name)
                build_graph(G, coauthor['scholar_id'], depth-1, bredth, last_author)

In [ ]:
G = nx.DiGraph()
output_path = '%s_%d_%d.gexf'%(name.replace(' ','_'), depth, bredth)
build_graph(G, sch_id, depth, bredth)
nx.write_gexf(G, output_path)